In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
from ipywidgets import *
import numpy as np
import k3d
import matplotlib.pyplot as plt
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *
from sloppy.metrics import *
from cavities import *
from functools import partial
from scipy.optimize import minimize, minimize_scalar
import pandas as pd

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\joptic.py:59: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [3]:
import warnings; warnings.simplefilter('ignore')

In [4]:
parname = 'lens_dist'

In [5]:
#defnition of (derived) cavity designs
#since default values are used to get default parameters, proxy functions are defined for 10mm etc. designs

def Cav2L2QInside10(arm1=110., arm2=53., base=43., angle=110., lens_dist=49.0, Rlens=10.0, quart_thick=2., c4=0., c6=0., c8=0.):
    return Cav2L2QInside(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, Rlens=Rlens, quart_thick=quart_thick, c4=c4, c6=c6, c8=c8)

def Cav2L2QOutside10(arm1=120., arm2=58., base=47., angle=110., lens_dist=47.0, Rlens=10.0, quart_thick=2., c4=0., c6=0., c8=0.):
    return Cav2L2QOutside(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, Rlens=Rlens, quart_thick=quart_thick, c4=c4, c6=c6, c8=c8)

def Cav4L1Qflip10(arm1=41., arm2=21.5, base=19., angle=110., lens_dist=28.9, Rlens=10.0, quart_thick=2., c4=0., c6=0., c8=0.):
    return Cav4L1Qflip(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, Rlens=Rlens, quart_thick=quart_thick, c4=c4, c6=c6, c8=c8)

def Cav4L1Q10(arm1=41., arm2=21.5, base=19., angle=110., lens_dist=28.9, Rlens=10.0, quart_thick=2., c4=0., c6=0., c8=0.):
    return Cav4L1Q(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, Rlens=Rlens, quart_thick=quart_thick, c4=c4, c6=c6, c8=c8)

def Cav2A10(arm1=120., arm2=58., base=47., angle=110., lens_dist=47.1, Rlens=10.0, c4=coeffc4(5.0), c6=coeffc6(5.0), c8=coeffc8(5.0)):
    return Cav2A(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, Rlens=Rlens, c4=c4, c6=c6, c8=c8)

def Cav4A10(arm1=44.8, arm2=23.6, base=20.8, angle=110., lens_dist=27.4, Rlens=10.0, c4=0., c6=0., c8=0.):
    return Cav4A(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, Rlens=Rlens, c4=c4, c6=c6, c8=c8)

def Cav4Aflip10(arm1=41., arm2=21.5, base=19., angle=110., lens_dist=28.8, Rlens=10.0, c4=0., c6=0., c8=0.):
    return Cav4Aflip(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, Rlens=Rlens, c4=c4, c6=c6, c8=c8)

In [6]:
allcavs = {'Cav2L2QInside': {'fct': Cav2L2QInside, 'x0': [3.1e-3, 9e-5], 'bounds': [(2.2e-3, 4e-3), (0., 3e-4)], 'dgmn': 1},\
            'Cav2L2QOutside': {'fct': Cav2L2QOutside, 'x0': [7.015e-04, 2.16e-05], 'bounds': [(0., 1e-3), (0., 3e-4)], 'dgmn': 1},\
            'Cav2L2QInside10': {'fct': Cav2L2QInside10, 'x0': [1.7e-04, 6.0e-08], 'bounds':  [(0., 1e-3), (-1e-5, 1e-5)], 'dgmn': 1},\
            'Cav2L2QOutside10': {'fct': Cav2L2QOutside10, 'x0': [8.066e-05, 5.60e-07], 'bounds':  [(1e-5, 1e-4), (-1e-6, 1e-6)], 'dgmn': 1},\
            'Cav4L1Qflip10': {'fct': Cav4L1Qflip10, 'x0': [0.00727, 0.00752], 'bounds':  [(0, 2e-2), (-1e-2, 1e-2)], 'dgmn': 1},\
            'Cav4L1Q10': {'fct': Cav4L1Q10, 'x0': [0.00619342, 0.00029877], 'bounds': [(0., 1e-2), (-1e-3, 1e-3)], 'dgmn': 1},\
            'TetCav2L1Q': {'fct': TetCav2L1Q, 'x0': [9.1e-4, 4.30e-05], 'bounds': [(0., 2e-3), (-1e-4, 1e-4)], 'dgmn': 2},\
            'Cav2A10': {'fct': Cav2A10, 'x0': [5.5e-05, -4.50e-06], 'bounds': [(0., 1e-4), (-1e-5, 1e-5)], 'dgmn': 1},\
            'Cav4A10': {'fct': Cav4A10, 'x0': [1.7449e-05, -4.30e-07], 'bounds': [(0., 1e-4), (-1e-5, 1e-5)], 'dgmn': 1},\
            'Cav4Aflip10': {'fct': Cav4Aflip10, 'x0': [9.205e-06, -5.996e-07], 'bounds': [(0., 1e-4), (-1e-5, 1e-5)], 'dgmn': 1}}

In [7]:
c4s = []
c6s = []
fsrs = []
waists = []
scores = []
parname = 'lens_dist'

i=0
for k, dct in allcavs.items():
    cavfct = dct['fct']
    print(cavfct.__name__)
    res = degeneracy_length(cavfct, parname, 1e-1)
    Ldeg=res.x
    #optimize
    fun = partial(metric_optim, metr=metric_var, cavfct=cavfct, cavpars={parname: Ldeg}, rmax=8., degmodenum=dct['dgmn'])
    bounds = dct['bounds']
    x0 = dct['x0']
    res = minimize(fun, x0=x0, bounds=bounds)
    print(res.x)
    plot_metrics(cavfct, cavpars={parname: Ldeg, 'c4': res.x[0], 'c6': res.x[1]}, s=1, a=np.arange(1,10), degmodenum=dct['dgmn'])
    sys = RaySystem(cavfct(**{parname: Ldeg, 'c4': res.x[0], 'c6': res.x[1]}))
    system = sys.abcd
    fsrs.append( system.fsr )
    waists.append( system.waist_at(0)[0] )
    c4s.append( res.x[0] )
    c6s.append( res.x[1] )
    scores.append(res.fun)
    i += 1

Cav2L2QInside
[3.09159012e-03 8.99540817e-05]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Cav2L2QOutside
[7.01530535e-04 2.16285945e-05]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Cav2L2QInside10
[1.7e-04 6.0e-08]


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Cav4A10
[ 1.74488429e-05 -4.30086189e-07]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Cav4Aflip10
[ 9.205e-06 -5.996e-07]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
d = {'name': list(allcavs.keys()), 'FSR [MHz]': np.array(fsrs)*1e-6, 'w [mm]': np.array(waists)*1e3, 'c4 [mm^-3]': c4s, 'c6 [mm^-5]': c6s, 'score [au]': scores}

df = pd.DataFrame(data=d)

In [22]:
df

,name,FSR [MHz],w [mm],c4 [mm^-3],c6 [mm^-5],score [au]
0,Cav2L2QInside,2281.399258,19.915692,0.003092,8.995408e-05,0.007014
1,Cav2L2QOutside,2281.399258,19.915692,0.000702,2.162859e-05,0.008786
2,Cav2L2QInside10,906.825792,22.744117,0.000170,6.000000e-08,NaN
3,Cav2L2QOutside10,831.522138,21.357559,0.000081,5.589356e-07,0.006116
4,Cav4L1Qflip10,2199.407680,19.921522,0.007270,7.520000e-03,NaN
5,Cav4L1Q10,2199.407680,21.765187,0.005998,1.000000e-03,0.072446
6,TetCav2L1Q,2302.235637,20.700861,0.000861,7.492982e-05,0.024496
7,Cav2A10,835.728894,21.305293,0.000055,-4.500000e-06,NaN
8,Cav4A10,2046.427690,19.832230,0.000017,-4.300862e-07,0.008227
9,Cav4Aflip10,2229.086092,19.741928,0.000009,-5.996000e-07,0.008553
